# Project Title

## Overview

The International Union for Conservation of Nature (IUCN) created the “Red List of Threatened Species” in 1964, which according to [their website](https://www.iucnredlist.org/) is “the world’s most comprehensive information source on the global conservation status of animal, fungi and plant species.” The IUCN maintains data pertaining to over 138,300 species in order to determine whether or not they are threatened with global extinction. Given the significant amount of labor it takes to maintain data on all of these species, our firm’s goal is to help the IUCN better predict which species will soon be at risk of extinction, which will hopefully help them direct their efforts in a more effective and targeted fashion. We hope to help the IUCN with all species, but in order to demonstrate our firm’s capabilities, we are focusing specifically on data pertaining to bird species for the purposes of this notebook.

## Business Problem

Our task is analyzing data pertaining to birds in order to assess whether or not each species is ‘threatened” or “not threatened” with global extinction. This information is valuable to the IUCN given that they will be able to direct their conservation efforts toward those species at greater risk of extinction. The Red List is updated every year for a subset of bird species with an extensive overhaul of each species every 4 years, so our hope is that our model will help the IUCN be able to direct their conservation efforts as effectively as possible. 

The IUCN Red List uses these 9 categories for all species to assess their threat level:

1) Not Evaluated (NE)\
2) Data Deficient (DD)\
3) Least Concern (LC)\
4) Near Threatened (NT)\
5) Vulnerable (VU)\
6) Endangered (EN)\
7) Critically Endangered (CR)\
8) Extinct in the Wild (EW)\
9) Extinct (EX)

The IUCN Red List guidelines seem to err on the side of caution when making their assessments ([see Table 2.1 on p. 16 of the attached PDF](https://cmsdocs.s3.amazonaws.com/RedListGuidelines.pdf)). This is understandable given that it is likely easier to save a great number of species from extinction if more species are considered “threatened.” 

With this in mind, our model will consider all labels from “vulnerable” through “extinct” to indicate that the species is threatened, with “near threatened” and “least concern” being categorized as “not threatened.” The category “not evaluated” is not included in the data set given that the IUCN has not included data on these species. 

“Data Deficient” represents species that have insufficient data to make a determination regarding their threat status. One of our firm’s goals is to use our model on the 41 bird species in this category in order to predict whether or not they are at risk of extinction to hopefully lead the IUCN to consider these species as part of the “threatened” group despite the fact that they are not labeled as such.

Given that the IUCN tends to err on the side of caution when making their assessments, we have decided it would be in our best interest to follow suit. In this sense, we will optimize our models for “recall” over “precision,” in that we are interested in minimizing the number of false negatives rather than false positives. In the context of this business problem, a false negative would be represented by identifying a species as not threatened when in reality they are, which means the relevant authorities could potentially miss the fact that these species are at risk of becoming extinct before it is too late. A false positive would be represented by identifying a species as threatened when in reality they are not, which might mean allocating resources to conserving this species that are not necessarily needed. While this is also potentially a problem, our understanding is that this is a less serious problem than false negatives in this context.


## Data Understanding

The data used in this notebook was initially prepared by two of our colleagues, Jeff Marvel and Crystal Gould Perrott [find their GitHub repository here](https://github.com/marvelje/bird-data-exploration). We made edits to this data which will be described in more detail below, but here are descriptions of the original data sources in the order they appear in the original data set:

- [Ecological Society of America](https://figshare.com/collections/EltonTraits_1_0_Species-level_foraging_attributes_of_the_world_s_birds_and_mammals/3306933)
    - Information pertaining to family name, diet, foraging strategy, and body mass.
        - The family names are strings that identify each species in Latin and English. Body mass is a measurement of each species’s average weight in grams. The other categories were already formatted in a binary fashion by our colleagues Jeff and Crystal, indicating that each bird species either does or does not belong to a particular category.
- [Bird Life International](https://www.birdlife.org/)
    - Information pertaining to family name, Global IUCN Red List Category, endemic breeding status, migratory status, region(s) in which the bird is primarily found, threat types facing each bird species, and the terrain type where each species is primarily found.
        - The Global IUCN Red List Category is a categorical column containing one of the above 8 labels corresponding to each bird’s threatened status. This is the data we used as our target variable for all models run in the notebook. In order to make this a binary feature, we binned this data so that “threatened” species correspond to a “1” (includes VU, EN, CR, EW, and EX labels) while “not threatened” species correspond to a “0” (includes NT and LC labels). As stated above, we left DD species out of our modeling so that we could predict their status later using our final model, and NE species are not represented in this data set.
        - The family names are strings that identify each species in Latin and English. The remaining columns (endemic breeding, migratory status, regions, threat types, and terrain types) were already formatted in a binary fashion by our colleagues Jeff and Crystal, indicating that each bird species either does or does not belong to a particular category.
- It is important to note that there is an imbalance in our data when considering the target variable (threatened/not threatened). Based on how we binned the IUCN Global Red List Category, roughly 13% of the bird species in this data set are considered “threatened” with the remaining 87% being “not threatened.” 

In [ ]:
# code here to explore your data

## Data Preparation

- Based on our business understanding, we decided to drop the following 18 columns from the data set given that they were not going to impact our analysis:
    - 'SpecID', 'IOCOrder', 'BLFamilyLatin', 'BLFamilyEnglish', 'BLFamSequID', ‘Taxo', 'Scientific', 'English', 'Diet-Source', 'Diet-EnteredBy', ‘ForStrat-Source', 'ForStrat-EnteredBy', 'BodyMass-Comment', 'BodyMass-Source', 'Record-Comment', 'Unnamed: 0.1', 'English name', 'Family'
- We created a new data frame containing all of the 41 bird species labeled as “DD” for testing later on. These 41 rows were then dropped from our set before doing a train-test split.
- We also created a new column ‘threatened’ with a binary label pertaining to each species threatened status as indicated by the Global IUCN Red List Category data. All LC and NT labels were given a 0 for “not threatened,” while all VU, EN, CR, EW, and EX were given a 1 for “threatened.” 

In [1]:
# code here to prepare your data

## Modeling

- Our process involved a 75/15/10 split of the data set. 75% was used for training our model, which we then tested against 15% of the data. The remaining 10% was used as a holdout set and was not involved in any of our modeling—this data was only used for testing our final model to verify our results.
- To make our evaluation process more efficient, we set up some pipelines to deal with different category types. 
    - Numerical: 
        - This only applied to the body mass column, given that this was a variable containing information about each bird species’s average mass in grams. For this column, we applied a MinMaxScaler in order to set this data on a scale from 0 to 1, which would more effectively correspond to the binary 0 and 1 data in the rest of our columns.
    - Proportional:
        - There are 10 different types of “diet,” and 7 different “foraging strategies” included in this data set. Given that these groups of columns each represent 100% of each species’s diet and foraging strategy, the individual columns include values between 0–100. In other words, for each species, you would need to add together all of the values in the 10 diet columns to determine the components of their diet that add up to the 100% total. Thus, we used a FunctionTransformer for these columns to divide all of these values by 100, again putting them on a scale from 0 to 1.
    - Categorical:
        - After deleting the 18 columns mentioned above, there were only three categorical columns remaining in the data set–PassNonPass, Diet-5Cat, and Diet-Certainty. For these columns, we used a OneHotEncoder to produce new columns that use 0s and 1s for each species to represent whether or not that species belongs to that category.

## Evaluation

- We ran a variety of model types, but in order to effectively compare our results we created an “evaluate” function.
    - This function takes as inputs: 
        - A model (in each instance for us this was a GridSearchCV)
        - Training and testing data 
        - A grid_search parameter 
    - The output of this function includes: 
        - Accuracy, precision, recall, and f1 scores 
        - The “best parameters” and a summary data frame of each model run in the grid search 
        - A confusion matrix, ROC-AUC curve, and Precision-Recall curve for the test data. 
- Throughout our modeling process, we used the following model types:
    - Logistic regression
    - Decision trees
    - KNN
    - Random Forest
    - Bagged Trees
    - Naive Bayes
    - Boosted Trees
- All comments regarding individual model evaluation will be included in markdown/comments in the cells below

### Baseline Understanding

- Given the imbalance in our data set discussed above, we can assume that a baseline model that only predicted “not threatened” for each bird species would be 87% accurate. 

In [ ]:
# code here to arrive at a baseline prediction

### First $&(@# Model

Before going too far down the data preparation rabbit hole, be sure to check your work against a first 'substandard' model! What is the easiest way for you to find out how hard your problem is?

In [ ]:
# code here for your first 'substandard' model

In [ ]:
# code here to evaluate your first 'substandard' model

### Modeling Iterations

Now you can start to use the results of your first model to iterate - there are many options!

In [ ]:
# code here to iteratively improve your models

In [ ]:
# code here to evaluate your iterations

### 'Final' Model

In the end, you'll arrive at a 'final' model - aka the one you'll use to make your recommendations/conclusions. This likely blends any group work. It might not be the one with the highest scores, but instead might be considered 'final' or 'best' for other reasons.

In [ ]:
# code here to show your final model

In [ ]:
# code here to evaluate your final model

## Conclusions

Our final model is an average of a logistic regression model only analyzing the threat types facing each species, and a decision tree model analyzing all data in our set. This model was tested on our 10% holdout set, which has led us to the following results and recommendations:
- Recall is 92%, which is good considering our business understanding of minimizing false negatives.
- Precision is 51%, which ADD MORE HERE
- Based on the results of our model, one recommendation for our stakeholder would be to prioritize conservation efforts on the species indicated as “threatened.” 
- After testing on the 41 “DD” species, our model predicts that 17 of these species are actually threatened. One recommendation for our stakeholder would be to consider these specific species as threatened rather than “DD” in order to hopefully begin conservation efforts before it is too late.
- We also analyzed the feature importance of the various “threat types” facing these bird species, given that these threat types seemed to have a greater impact on each species’s risk of extinction than other features in our data set. Of these threat types, we found that 1) Agriculture & aquaculture and 2) Invasive & other problematic species, genes & diseases are the most significant threat factors impacting the model. One recommendation for our stakeholder would be to focus conservation efforts on mitigating these threat types in particular.

## Limitations

Our data set includes information pertaining to 9597 bird species, but as indicated above, there are other bird species that have not been studied enough to include. While our data set does represent a significant percentage of all known bird species ([according to Bird Life](https://www.birdlife.org/birds/), there are more than 11,000 species), it is not exhaustive.

## Future Directions

There are a number of possible directions to go in the future, but here are some thoughts we have regarding possible ways to expand this initial study:
- The IUCN Red List takes a number of factors into account when assessing the threat level for each species, but two of the most important factors are species population size and the geographic range that species inhabits. With that in mind, we would like to collect additional data regarding population and geographic range over time so that we can conduct a time series analysis. Our hope is that analyzing these time series trends would give us a better sense of how soon each species might be at risk of extinction, which could be useful for our stakeholder in terms of prioritizing conservation efforts.
- Our model was able to identify which threat types are impacting the threatened status of each bird species. Thus, one possible future direction would be to collect more data pertaining to these threat types so that we can better understand what action(s) our stakeholder might take for more effective conservation efforts. 